In [3]:
import warnings; warnings.simplefilter('ignore')
import gym
import numpy as np
import tflearn
from tflearn.layers.core import input_data
from tflearn.layers.core import dropout
from tflearn.layers.core import fully_connected
from tflearn.layers.estimator import regression
from collections import Counter
import time
import random

In [4]:
LR = 1e-3
env = gym.make('CartPole-v1')
goal_steps = 500 # Max no. of time steps in each episode
score_requirement = 50
initial_games = 10000 # No. of episodes

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


initial_population() will create a training set using random action in each state (observation).

In [5]:
def initial_population():
    
    training_data = []
    all_scores = []
    accepted_scores = []
    
    for _ in range(initial_games): # Looping through all episodes
        
        score = 0 # Value
        game_memory = [] # Policy = [State, Action]
        prev_observation = env.reset() # Initial State
        
        for _ in range(goal_steps):  # Looping through all time_steps in the episode
            
            action = random.randrange(0,2)
            # action = env.action_space.sample() # Random action chosen from the action_space
            
            
            observation, reward, done, info = env.step(action)
            
            game_memory.append([prev_observation, action])
            prev_observation = observation
            score += reward
            
            if done:
                break
                
        
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                    
                training_data.append([data[0], output])
        
        
        all_scores.append(score)
        
        
    np.save('Data_Files/saved.npy', np.array(training_data))
    
    print('Average accepted score: {}'.format(np.mean(accepted_scores)))
    print('Median accepted score: {}'.format(np.median(accepted_scores)))
    print(Counter(accepted_scores))
    
    return training_data



#initial_population();
                

In [4]:
def neural_network_model(input_size):
    
    network = input_data(shape=[None, input_size, 1], name='input')
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR,\
                        loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')
    
    return model

    

In [1]:
def train_model(training_data, model=False):
    
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]), 1)
    y = [i[1] for i in training_data]
    
    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input':X}, {'targets':y}, n_epoch=3, snapshot_step=500, \
             show_metric=True, run_id='openai_learning')
    
    return model

In [2]:
training_data = initial_population()

NameError: name 'initial_population' is not defined

In [23]:
model = train_model(training_data)

---------------------------------
Run id: openai_learning
Log directory: log/
INFO:tensorflow:Summary name Accuracy_2/Adam (raw) is illegal; using Accuracy_2/Adam__raw_ instead.
---------------------------------
Training samples: 69042
Validation samples: 0
--


InvalidArgumentError: You must feed a value for placeholder tensor 'targets_1/Y' with dtype float and shape [?,2]
	 [[Node: targets_1/Y = Placeholder[dtype=DT_FLOAT, shape=[?,2], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'targets_1/Y', defined at:
  File "/home/paresh/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/paresh/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/paresh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/paresh/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/paresh/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/paresh/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", li

In [9]:
all_scores = []
choices = []

for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    
    for _ in range(goal_steps):
        #env.render()
        
        if len(prev_obs) == 0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
        
        choices.append(action)
        
        observation, reward, done, info = env.step(action)
        prev_obs = observation
        game_memory.append([observation, action])
        score += reward
        
        if done:
            break
            
    all_scores.append(score)
    
print('Average Score: {}'.format(sum(all_scores)/len(all_scores)))
print('Choices 1: {}, Choice 2: {}'.format(choices.count(1)/len(choices), \
                                           choices.count(0)/len(choices)))

Average Score: 306.0
Choices 1: 0.4980392156862745, Choice 2: 0.5019607843137255


In [10]:
# model.save('Model_Files/carpole_v1.model')

In [11]:
# model.load('Model_Files/carpole_v1.model')